# SANTANDER DATA MASTER - CIENTISTA DE DADOS

### Questão b) Classificar clientes de 1 a 5

Descrição: A segunda tarefa consiste em dar uma nota de 1 a 5 para cada cliente da base teste, respeitando a variável ‘TARGET’, isto é, o seu nível de satisfação, sendo 1 o mais insatisfeito e 5 o mais satisfeito. Ao dar essa nota deve-se ter em mente que
somente os clientes com nota 1 serão alvos de uma ação de retenção e que o objetivo dessa ação é maximizar o lucro esperado por cliente (usando os mesmos valores da primeira questão).

In [21]:
#Imports

#Manipulação dos Dados
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc
from imblearn.over_sampling import SMOTE
import xgboost as xgb



In [10]:
df_train = pd.read_pickle('./train_1.pickle')
df_test = pd.read_pickle('./test_1.pickle')

In [11]:
df_train.shape,df_test.shape

((76020, 142), (75818, 141))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['TARGET','ID'], axis = 1), df_train.TARGET, stratify = df_train.TARGET, train_size = 0.80 ,random_state = 42)

In [13]:
smote = SMOTE(sampling_strategy='minority')
X_over, y_over = smote.fit_resample(X_train,y_train)

In [14]:
X_over.shape, y_over.shape

((116820, 140), (116820,))

In [15]:
(y_over.value_counts()/y_over.count())*100

0    50.0
1    50.0
Name: TARGET, dtype: float64

In [17]:
# Generating the model with the optimized hyperparameters
clf_optimized = xgb.XGBClassifier(learning_rate = 0.007961566078062952, n_estimators = 1397, 
                                  max_depth = 4, min_child_weight = 5.711008778424264, 
                                  gamma = 0.2816441089227697, subsample = 0.692708251269958, 
                                  colsample_bytree = 0.5079831261101071, seed = 42)

In [18]:
# Fitting the model to the X_train_selected dataset
clf_optimized.fit(X_over, y_over)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.5079831261101071, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2816441089227697, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.007961566078062952, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=5.711008778424264,
              missing=nan, monotone_constraints='()', n_estimators=1397,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [25]:
y_predicted = clf_optimized.predict_proba(X_test)[:,1]
auc(y_test, y_predicted)

0.8067820887413139